In this document, I have done what any physics major would do seeing field lines and plotted the vector field behind it ( that is, the vector (d[cI]/dt, d[cro]/dt) at each sample point) for the phase diagram of [cI] and [cro] in the Ingalls model. I used Plotly for this, following examples from https://plotly.com/python-api-reference/generated/plotly.figure_factory.create_quiver.html to generate the vector field plot. I added that to my pre-existing interactive plot replicating the Ingalls graphic which can be found here: https://colab.research.google.com/drive/1zm9O4-tXva3ImRCRUXJlV6RNyDgxTEXd?usp=sharing
The details of the model are in the "New Ingalls replication: neat" note in the "ODE solvers" folder in my Onenote.

In [1]:
#Run for both figures
#testing scipy
from scipy.integrate import odeint
#imported for plotting
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objs as go
import plotly.figure_factory as ff
import math as math
# variable convention r is [cI], c is [cro], x=[r,c]

# expression rates
def fr(r,c):
    return (a+10*a*k1*((r/2)**2))/(1+k1*((r/2)**2)+k2*k1*((r/2)**3)+k3*(c/2)+k4*k3*((c/2)**2))
def fc(r,c):
    return (b+b*k3*(c/2))/(1+k1*((r/2)**2)+k2*k1*((r/2)**3)+k3*(c/2)+k4*k3*((c/2)**2))

#the model
def switch(x,t):
    r=x[0]
    c=x[1]
    return [fr(r,c)-δr*r,fc(r,c)-δc*c]

The cell below to get Figure A with vectors

In [2]:
#reproducing figure 7.11A with vectors
a=5
b=50
k1=1
k2=.1
k3=5
k4=.5
δr=.02 #lysogenic state; recA not degrading cI
δc=.02
state="A"

t=np.linspace(0, 1000, 100000)

initlistA=[[3,3],[5,25],[0,50],[10,250],[25,250],[50,250],[100,250],[150,250],[250,250],[250,150],[250,50]]
plotlistA=[]
for point0 in initlistA:
    plotlistA.append(odeint(switch,point0,t))

r,c = np.meshgrid(np.arange(0, 250, 5), np.arange(0, 250, 5))
u = (fr(r,c)-δr*r)
v = (fc(r,c)-δc*c)

fig = ff.create_quiver(r, c, u, v, scale=2, scaleratio=.8, angle=math.pi/6, line=dict(width=1), name='time derivative scaled by 2')

for i in range(0,len(plotlistA)):
    rlist=[]
    clist=[]
    k=0
    plotarray=plotlistA[i]
    for n in plotarray:
        if k%10==0:
            rlist.append(n[0])
            clist.append(n[1])
        k+=1
    fig.add_trace(go.Scatter(x=rlist, y=clist, mode='lines', name=f'path from {(rlist[0],round(clist[1]))}'))
fig.update_layout(
    title="Figure A (with time derivative vectors scaled by 2)",
    xaxis_title="[cI] (nM) (2d[cI]/dt (nM/s))",
    yaxis_title="[cro] (nM) (2d[cro]/dt (nM/s))")
fig.show()

Output hidden; open in https://colab.research.google.com to view.

Figure A models the state where RecA is inactive (a normal lysogenic cell). We can see clearly here how there are 2 attractors one at a steady cro concentration at about 100nM and no cI (the lytic path) and the other at a steady cI concentration of a little over 200 nM and no cro (stable lysogeny). For visibilty, the vectors have been scaled by a factor of 2 here.

Run the cell below to get Figure B with vectors

In [3]:
#reproducing figure 7.11B
#variables
a=5
b=50
k1=1
k2=.1
k3=5
k4=.5
δr=.2 #lytic state; recA degrading cI
δc=.02
state="B"

#time steps
t=np.linspace(0, 1000, 100000)

#inital conditions
initlistB=[[10,5],[8,8],[250,10],[250,60],[250,100],[250,150],[250,200],[250,250],[150,250],[50,250]]

#running the ODEs
plotlistB=[]
for point0 in initlistB:
    plotlistB.append(odeint(switch,point0,t))
plotlistB=[]
for point0 in initlistB:
    plotlistB.append(odeint(switch,point0,t))

x,y = np.meshgrid(np.arange(0, 250, 5), np.arange(0, 250, 5))
u = (fr(r,c)-δr*r)
v = (fc(r,c)-δc*c)

fig = ff.create_quiver(r, c, u, v, scale=.5, scaleratio=.8, angle=math.pi/6, line=dict(width=1),name='time derivative scaled by .5')
for i in range(0,len(plotlistB)):
    rlist=[]
    clist=[]
    k=0
    plotarray=plotlistB[i]
    for n in plotarray:
        if k%10==0:
            rlist.append(n[0])
            clist.append(n[1])
        k+=1
    fig.add_trace(go.Scatter(x=rlist, y=clist, mode='lines', name=f'path from {(rlist[0],round(clist[1]))}'))
fig.update_layout(
    title="Figure B (with time derivative vectors scaled by .5)",
    xaxis_title="[cI] (nM) (.5d[cI]/dt (nM/s))",
    yaxis_title="[cro] (nM) (.5d[cro]/dt (nM/s))")

fig.show()

Output hidden; open in https://colab.research.google.com to view.

Figure B models the state where RecA is active (SOS) and the decay rate of cI is increased by a factor of 10 when compared with normal lysogenic model in Figure A. We can see clearly here how the sole attractor is a steady cro concentration at about 100nM; the lytic path is the only option. For visibility, the vectors have been scaled by a factor of .5 here.